# Introduction to Data Engineering: Bridging the gap between raw data and insights

What we'll cover in this tutorial

In this tutorial, we'll be using SQLite, a simple and easy-to-use database, to explore the fundamentals of data engineering and data preprocessing. We'll cover the following topics:
+ Introduction to SQL: We'll learn the basics of SQL (Structured Query Language), the standard language for interacting with databases.
+ Data Loading: We'll load the transformed data into a new table or file for further analysis.
+ Data Extraction: We'll use SQL to extract data from our SQLite database.
+ Data Transformation: We'll use SQL to clean, transform, and prepare the data for analysis. This will include handling missing values, inconsistent formatting, and duplicates.


### Understanding SQL and Relational Databases

Relational databases organize data into tables, which you can visualize as spreadsheets with rows and columns. Each table represents a specific type of entity or concept – imagine tables for "Customers," "Products," or "Orders." 
To query these databases, we are using **SQL**, which is the standard language for interacting with relational databases. 

Rows (or Records): Each row in a table represents a single instance of that entity. For example, in the diagram below, in the "Order" table, each row would represent one specific order. We often refer to these as records.
Columns (Attributes): Each column in a table represents a characteristic or attribute of that entity. In the "Order" table, columns might include "UnitPrice," "Quantity," and "Discount." Each column has a specific data type (e.g., text, number, date).
The power of relational databases lies in their ability to represent relationships between different entities. These relationships are created using keys:

+ Primary Key: A unique identifier for each record within a table. For example, "CustomerID" in the "Customers" table would likely be a primary key, ensuring that each customer has a unique identifier.
+ Foreign Key: A column in one table that refers to the primary key of another table. This establishes a link between the two tables. For example, an "Orders" table might have a "CustomerID" column that is a foreign key referencing the "CustomerID" (primary key) in the "Customers" table. This way, you know which customer placed each order.

Tables are linked together through these keys, allowing you to perform complex queries that combine data from multiple tables. They can be linked using different types of relationships, such as one-to-one, one-to-many, or many-to-many.

Below is a database schema diagram that illustrates these concepts:


![Northwind Database Schema](./images/db-schema-northwind.svg)


---

#  We need to populate our database with some data !

In this tutorial, we will be using the Northwind database, a sample database that contains data about a fictional company that imports and exports specialty foods. It includes tables for customers, orders, products, and more.
The schema diagram above shows the relationships between the tables in the Northwind database.

We've created for you a small script that will create the Northwind database and populate it with some sample data. To make things easy, we use SQLite, a lightweight database that is easy to set up and use, and offers a simple way to work with SQL databases without needing a separate server.
In a real-world scenario, you would typically connect to a database server (like PostgreSQL, MySQL, Snowflake, etc...) and run SQL commands to create and populate your database. 

💡 **You don't really need to understand what's happening down there, you can just execute the code and move on to the next section.**

In [22]:
#Import required libraries for data processing

import urllib.request
import os
import sqlite3
import pandas as pd  # added for dataframe operations

In [23]:
# Verifying if the database file exists, if not, download it. Instantiate the connection to the database.

url = "https://github.com/jpwhite3/northwind-SQLite3/raw/refs/heads/main/dist/northwind.db"
db_path = "northwind.db"

if not os.path.exists(db_path):
    print("Downloading the database...")
    urllib.request.urlretrieve(url, db_path)
    print("Database downloaded successfully!")
else:
    print("Database already exists.")

# Connexion à la base
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


Database already exists.


In [24]:
def query_and_print(query):
    """
    Execute a SQL query and print the results.
    """
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Convert results to a DataFrame for better readability
    df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
    
    print(df.to_string(index=False))
    return df


---

# Back to our SQL queries

SQL is the standard language for talking to databases – the organized stores of information that power your business. Think of SQL as the tool that lets you ask questions of your data and get the answers you need to make better decisions. It's like having a direct line to your company's knowledge.

Why is SQL Important for Business Professionals?

SQL empowers you to:

Get answers to critical business questions: Instead of relying solely on reports or technical teams, you can use SQL to directly query your data and answer questions like:
+ What are my top-selling products this quarter?
+ Which marketing campaigns are generating the most leads?
+ What is the average order value for customers in a specific region?
+ Who are my most valuable customers, and what are they buying?




---

# 🤖 SQL + AI: Smarter Queries, Safer Data

AI speed up and simplify greatly the SQL query generation, but it requires a thoughtful approach, and a bit of practice to get the most out of it. Here are some tips to help you effectively use AI for SQL query generation:

### 1. Prompt Engineering:

Set the Stage: Begin by defining the context: "As a database engineer using [Your Database System]..." that will help the system understand the environment and constraints.
This important for the query to have the proper syntax and to avoid errors.

🫸 Schema as Context: Always provide the relevant database schema as text at the beginning of your prompt or discussion. Without it, the AI model cannot understand what data it is manipulating, and will NEVER give you 

### 2. Prompt Engineering:

Clarity is Key: Articulate your needs precisely. Avoid ambiguity!
Example Prompts:
"I want the products generating the most revenue between 2014 and 2018."
"I want the name and phone number of our lead customer with over $1 million in purchases."
Critical Review:

Always Validate the AI's Output!
Watch Out For:
+ Inaccurate dates or filters - That could get you wrong results and insights
+ Destructive commands like DELETE or DROP TABLE - This can put in danger your whole database, and compromise the operations of others !

---



---

# Let's cross the Rubicon !

Alright, let's get our hands dirty. Imagine we've just been hired as the new data analysts for a company called "Northwind traders". They are a small but ambitious company that imports and exports gourmet food products around the world.

Our manager has given us our first big mission: Help the company boost its sales.

That's a broad goal. Where do we even begin? As data people, our first instinct should be to look at the data we have. We need to understand the business before we can change it. Our main tool for this investigation will be SQL.

Let's start by asking some basic questions.


Peeking into the data with select and limit
Before we can think about sales, we need to know what we're actually selling. Our database has a table called Products that seems like a good place to start.

Let's ask the database to show us everything in that table. The command for "show me" in SQL is select. If we want to see every column, we use the asterisk *, which is a wildcard for "all".

In [25]:
query_and_print("SELECT * from Products;")

 ProductID                      ProductName  SupplierID  CategoryID      QuantityPerUnit  UnitPrice  UnitsInStock  UnitsOnOrder  ReorderLevel Discontinued
         1                             Chai           1           1   10 boxes x 20 bags      18.00            39             0            10            0
         2                            Chang           1           1   24 - 12 oz bottles      19.00            17            40            25            0
         3                    Aniseed Syrup           1           2  12 - 550 ml bottles      10.00            13            70            25            0
         4     Chef Anton's Cajun Seasoning           2           2       48 - 6 oz jars      22.00            53             0             0            0
         5           Chef Anton's Gumbo Mix           2           2             36 boxes      21.35             0             0             0            1
         6     Grandma's Boysenberry Spread           3           2   

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15.00,101,0,5,0
73,74,Longlife Tofu,4,7,5 kg pkg.,10.00,4,20,5,0
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,0
75,76,Lakkalikööri,23,1,500 ml,18.00,57,0,20,0


Running this, you'll see a lot of data. Maybe too much. We don't need to see every single product right now, we just want to get a feel for the table's structure.

To ask for just a sample, we can add a limit clause. This is useful for taking a quick peek without overwhelming our screen. Let's just look at the first 5 products. To do that, we will be using the clause "LIMIT".

In [26]:
query_and_print("SELECT * from Products LIMIT 5;")

 ProductID                  ProductName  SupplierID  CategoryID     QuantityPerUnit  UnitPrice  UnitsInStock  UnitsOnOrder  ReorderLevel Discontinued
         1                         Chai           1           1  10 boxes x 20 bags      18.00            39             0            10            0
         2                        Chang           1           1  24 - 12 oz bottles      19.00            17            40            25            0
         3                Aniseed Syrup           1           2 12 - 550 ml bottles      10.00            13            70            25            0
         4 Chef Anton's Cajun Seasoning           2           2      48 - 6 oz jars      22.00            53             0             0            0
         5       Chef Anton's Gumbo Mix           2           2            36 boxes      21.35             0             0             0            1


,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


Much better. Now we can see the columns clearly: ProductID, ProductName, SupplierID, UnitPrice, etc.

The * is handy, but most of the time we only care about a few specific columns. For our sales mission, the product name and its price are probably quite important. Let's select just those two columns.

In [27]:
query_and_print("select ProductName, UnitPrice from Products limit 5;")

                 ProductName  UnitPrice
                        Chai      18.00
                       Chang      19.00
               Aniseed Syrup      10.00
Chef Anton's Cajun Seasoning      22.00
      Chef Anton's Gumbo Mix      21.35


,ProductName,UnitPrice
0,Chai,18.00
1,Chang,19.00
2,Aniseed Syrup,10.00
3,Chef Anton's Cajun Seasoning,22.00
4,Chef Anton's Gumbo Mix,21.35


Here we go. We've just used the most fundamental command in SQL.

+ Select lets us choose the columns we want to see.
+ From tells the database which table to look in.
+ Limit restricts the number of rows returned.

### Finding what you need with where
Looking at a random list of products is a start, but to make smart decisions, we need to ask more specific questions. For instance, a simple sales strategy could be to promote our high-value items.

So, let's ask the database: "Show me the products that are expensive". We need to define "expensive". Let's say, for now, any product with a UnitPrice greater than 50.

To filter our data based on a condition, we use the where clause. Think of it as adding an "if" to your request.

In [28]:
query_and_print("select ProductName, UnitPrice from Products where UnitPrice > 50;");

            ProductName  UnitPrice
        Mishi Kobe Niku      97.00
       Carnarvon Tigers      62.50
 Sir Rodney's Marmalade      81.00
Thüringer Rostbratwurst     123.79
          Côte de Blaye     263.50
  Manjimup Dried Apples      53.00
   Raclette Courdavault      55.00


Now we have a list of our premium products. Notice we used the greater-than symbol >. You can use other familiar comparison operators too:

+ = : equals
+ < : less than
+ >= : greater than or equal to
+ <= : less than or equal to
+ <> or != : not equal to

👉 Let's try another one. What if we want to find information about a specific customer? Let's look at the Customers table and find all of our customers based in Germany. When we filter on text (also called a 'string'), we need to put the value in single quotes ' '.

In [29]:
query_and_print("select CompanyName, ContactName, City from Customers where Country = 'Germany';");

             CompanyName         ContactName           City
     Alfreds Futterkiste        Maria Anders         Berlin
 Blauer See Delikatessen          Hanna Moos       Mannheim
Drachenblut Delikatessen        Sven Ottlieb         Aachen
          Frankenversand       Peter Franken        München
         Königlich Essen       Philip Cramer    Brandenburg
     Lehmanns Marktstand      Renate Messner Frankfurt a.M.
  Morgenstern Gesundkost     Alexander Feuer        Leipzig
      Ottilies Käseladen Henriette Pfalzheim           Köln
              QUICK-Stop         Horst Kloss      Cunewalde
      Toms Spezialitäten       Karin Josephs        Münster
       Die Wandernde Kuh         Rita Müller      Stuttgart


#### Combining conditions with and and or

Our questions can get more complex. It's good that we found our expensive products, but what if an expensive product is out of stock? We can't sell it.

Let's refine our previous query. We want products that are both expensive (UnitPrice > 50) and actually in stock (UnitsInStock > 0).

To check for multiple conditions where all must be true, we use and.

In [30]:
query_and_print("select ProductName, UnitPrice, UnitsInStock from Products where UnitPrice > 50 and UnitsInStock > 0;");

           ProductName  UnitPrice  UnitsInStock
       Mishi Kobe Niku       97.0            29
      Carnarvon Tigers       62.5            42
Sir Rodney's Marmalade       81.0            40
         Côte de Blaye      263.5            17
 Manjimup Dried Apples       53.0            20
  Raclette Courdavault       55.0            79


This list is much more useful for a sales campaign.

Now, let's consider a different strategy. Perhaps we want to run a marketing campaign in our key European markets, say Germany and France. We need a list of customers who are in Germany or in France.

For this, we use the or operator. It will show a row if any of the conditions are met.

In [31]:
query_and_print("select CompanyName, Country from Customers where Country = 'Switzerland' or Country = 'France';");

              CompanyName     Country
 Blondesddsl père et fils      France
                 Bon app'      France
        Chop-suey Chinese Switzerland
          Du monde entier      France
        Folies gourmandes      France
      France restauration      France
     La corne d'abondance      France
         La maison d'Asie      France
        Paris spécialités      France
       Richter Supermarkt Switzerland
     Spécialités du monde      France
     Victuailles en stock      France
Vins et alcools Chevalier      France


### Creating new information with calculations
Let's look at the Order Details table. It seems to hold the key to our sales performance. It has OrderID, ProductID, UnitPrice, and Quantity.

Wait a minute 💡. It shows the price of a single unit and how many units were sold, but it doesn't have a column for the total value of that line item. No problem, we can calculate it ourselves, directly in the select statement.

In [34]:
# The 3 consecutive double quotes allow us to write a multi-line string in Python and to ignore quotes inside the string.
query_and_print("""select OrderID, ProductID, UnitPrice, Quantity, UnitPrice * Quantity
from "Order Details" limit 10;""")

 OrderID  ProductID  UnitPrice  Quantity  UnitPrice * Quantity
   10248         11       14.0        12                 168.0
   10248         42        9.8        10                  98.0
   10248         72       34.8         5                 174.0
   10249         14       18.6         9                 167.4
   10249         51       42.4        40                1696.0
   10250         41        7.7        10                  77.0
   10250         51       42.4        35                1484.0
   10250         65       16.8        15                 252.0
   10251         22       16.8         6                 100.8
   10251         57       15.6        15                 234.0


,OrderID,ProductID,UnitPrice,Quantity,UnitPrice * Quantity
0,10248,11,14.0,12,168.0
1,10248,42,9.8,10,98.0
2,10248,72,34.8,5,174.0
3,10249,14,18.6,9,167.4
4,10249,51,42.4,40,1696.0
5,10250,41,7.7,10,77.0
6,10250,51,42.4,35,1484.0
7,10250,65,16.8,15,252.0
8,10251,22,16.8,6,100.8
9,10251,57,15.6,15,234.0



---

# Can't beat'em? JOIN'em!

So, we've hit our first real data-wrangling challenge. We can see a ProductID in our list of valuable orders, but that's just a number. It doesn't tell our sales team whether the customer bought "Chai" or "Chef Anton's Gumbo Mix". 

That important piece of information, the product's name, lives in a completely different table, the Products table. This is intentional. 

**Databases are designed to be tidy** - instead of repeating the product name and description in every single order line, they just use a unique ID as a reference. It's like having a contact list with just phone numbers, and a separate address book. To find someone's address, you need to use their name to look them up in the other book. 

In SQL, this process of looking up and combining information from multiple tables is called a join. It's how we connect the dots using a shared piece of information—a "key" like ProductID—to build a more complete picture.

